In [1]:
import os, sys, inspect
import csv

import numpy as np
import pandas as pd


import matplotlib
from matplotlib import pyplot as plt

import collections
import time
import datetime

import re

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

working_dir = os.path.dirname(
    os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(working_dir)
sys.path.insert(0, parent_dir)

from scripts_python import scraper as scr

%load_ext autoreload
%autoreload 2

1. Chrome - Help - About Google Chrome
2. check version; e.g. `Version 79.0.3945.117 (Official Build) (64-bit)`
3. [download relevant ChromeDriver](https://chromedriver.chromium.org/downloads)

In [2]:
with open('../ignorabilia/glassdoor_scraper.csv', mode='r') as f_in:
    credentials = {k: v for k, v in csv.reader(f_in)}

In [3]:
browser = scr.get_web_browser()
browser.implicitly_wait(10)
scr.load_main_page(browser=browser)

scr.accept_cookies(browser=browser)

scr.sign_in(
    browser=browser,
    e_mail=credentials['e_mail'],
    password=credentials['password'])

https://www.glassdoor.co.uk


In [4]:
scr.do_search(
        browser=browser,
        keywords='data scientist',
        location='Paris (France)')

In [5]:
scr.set_filters(
        browser=browser,
        jobtype='fulltime',
        post_age='7',
        radius='5',
        )